In [1]:
import pandas as pd

# reads file as csv
df = pd.read_csv('clause_data/contract_clauses.csv', sep="|")

# keeps only valid ACTs
df = df.loc[(df['acordo'] == 1)&(df['extrato'] == 1)&(df['validity'] == 1)]
df = df.dropna()

# reindexes the dataframe with the default integer index
df = df.reset_index(drop=True)

print(len(df.index))
df.head()

601449


,contract_id,acordo,extrato,validity,name,subgroup,text
0,2009_055971,1,1,1,Minimum wage,Wage adjustment,Ficará garantido ao empregado motorista o valo...
1,2009_055971,1,1,1,Salary payment - means and timeframes,Wage payment,"Para as funções de motorista de carreta, bi-tr..."
2,2009_055971,1,1,1,Salary deductions,Wage adjustment,"Qualquer multa por excesso de velocidade, por ..."
3,2009_055971,1,1,1,Food assistance,Assistances,Os empregados motoristas externos receberão me...
4,2009_055971,1,1,1,Food assistance,Assistances,Levando-se em conta a crise econômica e a redu...


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt')

# stop words
stop_words = set(stopwords.words('portuguese'))

# adds custom stop words
custom_stop_words = ['parágrafo', 'nº', 'i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x',
                     'xi', 'xii', 'xiii', 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx', 'xxi',
                     'xxii', 'xxiii', 'xxiv', 'xxv', 'xxvi', 'xxvii', 'xxviii', 'xxix', 'xxx']
stop_words.update(custom_stop_words)

# stemmer
stemmer = SnowballStemmer('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/calvineng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/calvineng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import re
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class CBApreprocess(BaseEstimator, TransformerMixin):
    def __init__(self, preprocess=True, remove_punctuation=True,
                 replace_numbers=True, remove_stopwords=True, stemming=True):
        self.preprocess = preprocess
        self.remove_punctuation = remove_punctuation
        self.replace_numbers = replace_numbers
        self.remove_stopwords = remove_stopwords
        self.stemming = stemming
        
    def preprocess_text(self, text):
        if self.remove_punctuation:
            text = re.sub(r'[^\w\s]|º', '', text)
        if self.replace_numbers:
            text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
        if self.remove_stopwords:
            words = text.split()
            words = [word for word in words if word.lower() not in stop_words]
            text = ' '.join(words)
        if self.stemming and stemmer is not None:
            words = text.split()
            words = [stemmer.stem(word) for word in words]
            text = ' '.join(words)
        return text.lower()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = [
            self.preprocess_text(cba)
            for cba in X
        ]
        return np.array(X_transformed)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

class CBAToTFIDFTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
    def fit(self, X, y=None):
        self.vectorizer.fit(X)
        return self
    def transform(self, X, y=None):
        X_tfidf = self.vectorizer.transform(X)
        return X_tfidf.toarray()

In [5]:
import spacy

# lemmatizer
nlp = spacy.load("pt_core_news_sm")

# stop words
stop_words = nlp.Defaults.stop_words

# adds custom stop words
custom_stop_words = ['parágrafo', 'nº', 'i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x',
                     'xi', 'xii', 'xiii', 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx', 'xxi',
                     'xxii', 'xxiii', 'xxiv', 'xxv', 'xxvi', 'xxvii', 'xxviii', 'xxix', 'xxx']
stop_words.update(custom_stop_words)

In [6]:
class CBAPreprocessLemmatize(BaseEstimator, TransformerMixin):
    def __init__(self, preprocess=True, remove_punctuation=True,
                 replace_numbers=True, remove_stopwords=True, lemmatize=True):
        self.preprocess = preprocess
        self.remove_punctuation = remove_punctuation
        self.replace_numbers = replace_numbers
        self.remove_stopwords = remove_stopwords
        self.lemmatize = lemmatize
        
    def preprocess_text(self, text):
        if self.remove_punctuation:
            text = re.sub(r'[^\w\s]|º', '', text)
        if self.replace_numbers:
            text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
        if self.remove_stopwords:
            words = text.split()
            words = [word for word in words if word.lower() not in stop_words]
            text = ' '.join(words)
        if self.lemmatize:
            doc = nlp(text)
            words = [token.lemma_ for token in doc]
            text = ' '.join(words)
        return text.lower()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = [
            self.preprocess_text(cba)
            for cba in X
        ]
        return np.array(X_transformed)

In [7]:
from sklearn.pipeline import Pipeline

# creates Pipeline to preprocess and calculate TF-IDF
preprocess_pipeline = Pipeline([
    ("cba_preprocess", CBApreprocess()),
    ("cba_to_tfidf", CBAToTFIDFTransformer())
])

# number of words to output and groupings
groups = df.groupby('subgroup')
n_top_words = 20

for name, group in groups:
    data_texts = group['text'].tolist()
    data_tfidf = preprocess_pipeline.fit_transform(data_texts)
    mean_tfidf = data_tfidf.mean(axis=0)
    top_indices = np.argsort(mean_tfidf)[-n_top_words:]
    vocab = preprocess_pipeline.named_steps['cba_to_tfidf'].vectorizer.get_feature_names_out()
    top_words = [vocab[idx] for idx in top_indices]
    print(f"Top {n_top_words} words for '{name}': {top_words}")

Top 20 words for 'Assistances': ['acord', 'fic', 'pagament', 'dependent', 'benefíci', 'conced', 'rea', 'val', 'auxíli', 'aliment', 'pag', 'cas', 'transport', 'ser', 'fornec', 'salári', 'trabalh', 'empres', 'empreg', 'numb']
Top 20 words for 'Bonuses': ['ano', 'receb', 'caix', 'promoçã', 'funçã', 'adiant', 'gratific', 'cinquent', 'pagament', 'parcel', 'dia', 'dias', 'pag', 'trabalh', 'ser', 'fér', 'empres', 'salári', 'empreg', 'numb']
Top 20 words for 'Contract types': ['estagiári', 'acord', 'lei', 'aprendizag', 'períod', 'mesm', 'funçã', 'superior', 'estági', 'praz', 'ser', 'salári', 'dias', 'aprendiz', 'trabalh', 'empres', 'experient', 'empreg', 'contrat', 'numb']
Top 20 words for 'Employment protections': ['just', 'provisór', 'afast', 'anos', 'gestant', 'salári', 'períod', 'aposentador', 'milit', 'praz', 'assegur', 'fic', 'empres', 'trabalh', 'estabil', 'garant', 'servic', 'dias', 'numb', 'empreg']
Top 20 words for 'General provisions': ['dirim', 'descumpr', 'justic', 'salári', 'esta

: 

: 